In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from openfast_io.turbsim_file import TurbSimFile
from openfast_io.turbsim_util import TurbsimReader, TurbsimWriter


In [2]:

hurrCat = 'CAT2'
numSeeds = 36
writeFolder = '../turb_sim/TurbSimGenFiles'

baseInputFolder = '../turb_sim'

# change this to your path to TurbSim executable
TurbSimExe = '../../i_devFiles/openfast/build/modules/turbsim/turbsim'

In [3]:
# functions
def turbSimSeedGen(baseInputFile, runDir, nSeeds=36, seed=0):
    # Reading turbsim input file
    ts_reader = TurbsimReader()
    ts_reader.read_input_file(baseInputFile)

    # if write folder does not exist, create it
    if not os.path.exists(runDir):
        os.makedirs(runDir)

    # need to copy the user defined spectrum file to the run directory
    if ts_reader.UserFile != '':
        shutil.copy(os.path.join(os.path.dirname(baseInputFile), ts_reader.UserFile), runDir)
    
    if ts_reader.ProfileFile != '':
        shutil.copy(os.path.join(os.path.dirname(baseInputFile), ts_reader.ProfileFile), runDir)



    btsFiles = []

    # Generating seeds
    for i in range(nSeeds):
        ts_reader.RandSeed1 = seed + i
        ts_writer = TurbsimWriter(ts_reader)
        fileName = os.path.join(runDir, f'turbInput_{i+1}.inp')

        ts_writer.execute(fileName)

        btsFiles.append(f'{fileName[:-4]}.bts')

    return btsFiles



In [4]:
# Generate the turbSim .bts files based on selected hurricane category
writeFolder = os.path.join(writeFolder, hurrCat)
btsFiles = turbSimSeedGen(os.path.join(baseInputFolder, f'TurbSim_{hurrCat}.inp'), f'{writeFolder}/withoutRamp', nSeeds = numSeeds)

# runing the TurbSim executable 
for i in range(numSeeds):
    os.system(f'{TurbSimExe} {btsFiles[i]}')

In [ ]:
for id, btsFile in enumerate(btsFiles):

    tsMod2 = TurbSimFile(btsFile)
    print(tsMod2)

    time4Ramp = 400.0 # seconds
    dt = tsMod2['dt']
    nRamp = int(time4Ramp / dt)

    iy, iz = tsMod2.closestPoint(y=0, z=150)

    print(nRamp)

    # Compute the ramp required to get to the desired wind speed
    ramp = np.linspace(3.0, tsMod2['u'][0,0,iy,iz], nRamp)


    # Create a new wind field with the ramp
    tempWind = np.zeros(shape=(3, nRamp, tsMod2['y'].size, tsMod2['z'].size), dtype=float)


    for idx in range(ramp.size):
        tempWind[0,idx,:,:] = ramp[idx]
        tempWind[1,idx,:,:] = 0.0
        tempWind[2,idx,:,:] = 0.0

    tsMod2['t'] = np.concatenate((np.linspace(0.0, time4Ramp, nRamp), tsMod2['t'] + time4Ramp), axis=0)
    tsMod2['u'] = np.concatenate((tempWind, tsMod2['u']), axis=1)

    # reshaping the tower wind field
    tsMod2['uTwr'] = tsMod2['uTwr'].reshape(3,tsMod2['t'].size,0)

    # print(tsMod2)

    tsMod2['ID'] = 7 # Changing to non periodic

    # assuming we are reading from the "withoutRamp" folder
    tsMod2.write(f'{os.path.join(os.path.dirname(os.path.dirname(btsFile)), os.path.basename(btsFile)[:-4] + "_ramp.bts")}')

    # --- Extract a horizontal plane and plot it
    U, V, W = tsMod2.horizontalPlane(z=150)
    T, Y = np.meshgrid(tsMod2.t, tsMod2.y)
    fig,ax = plt.subplots(1, 1)
    cs = ax.contourf(T, Y, U.T)
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('y [m]')
    ax.set_title('Velocity at z=150')
    plt.colorbar(cs)

    plt.savefig(f'{os.path.join(os.path.dirname(os.path.dirname(btsFile)), os.path.basename(btsFile)[:-4] + "_ramp.png")}')